# Run this cell first

In [ ]:
# this code enables the automated feedback. If you remove this, you won't get any feedback
# so don't delete this cell!
try:
  import AutoFeedback
except (ModuleNotFoundError, ImportError):
  !pip install AutoFeedback
  import AutoFeedback

try:
  from testsrc import test_main
except (ModuleNotFoundError, ImportError):
  !pip install "git+https://github.com/autofeedback-exercises/exercises.git@main#subdirectory=New-MTH4332/LennardJonesII"
  from testsrc import test_main

def runtest(tlist):
  import unittest
  from contextlib import redirect_stderr
  from os import devnull
  with redirect_stderr(open(devnull, 'w')):
    suite = unittest.TestSuite()
    for tname in tlist:
      suite.addTest(eval(f"test_main.UnitTests.{tname}"))
    runner = unittest.TextTestRunner()
    try:
      runner.run(suite)
    except AssertionError:
      pass


This next cell loads some code that allows us to use ase to run simulations of systems with a pair potential of your own design.  In this notebook, that pair potential is the Lennard Jones potential.  If you want to run simulations like those that we have performed here but using some other potential it may be useful for you to copy and reuse the code in this cell.  

_This is the same code that we loaded here in the exercise on running MD with a pair potential._

In [ ]:
import ase
import numpy as np
import scipy.stats
import matplotlib.pyplot as plt
from ase.build import bulk,make_supercell
from ase.visualize import view
from ase.io import write
from ase.neighborlist import NeighborList
from ase.calculators.calculator import Calculator, all_changes
from ase.stress import full_3x3_to_voigt_6_stress
from ase.lattice.cubic import FaceCenteredCubic
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
from ase.md.langevin import Langevin
from ase.io.trajectory import Trajectory 

class pairwise_calculator(Calculator) :
  """Implementation of a very basic Lennard Jones Calculator"""

  implemented_properties = ['energy', 'energies', 'forces', 'free_energy']
  implemented_properties += ['stress', 'stresses']  # bulk properties
  default_parameters = {'rc': None,}
  nolabel = True

  def __init__(self, **kwargs ) :
    Calculator.__init__( self, **kwargs )
    if self.parameters.pairwise_e is None :
      ValueError("function for evaluating pairwise energies has not been set")
    # Setup a cutoff value so we can use a neighbour list
    if self.parameters.rc is None : 
      ValueError("cutoff for pairwise interactions should be set")
    # Setup stuff for neighbour list
    self.nl = None
    self.pairwise_e = self.parameters.pairwise_e
    self.parameters.pairwise_e = None

  def calculate( self, atoms=None, properties=None, system_changes=all_changes, ) :
    if properties is None : properties = self.implemented_properties

    Calculator.calculate(self, atoms, properties, system_changes)

    natoms = len(self.atoms)

    rc = self.parameters.rc
    # potential value at rc
    e0, f0 = self.pairwise_e( rc )     

    if self.nl is None or 'numbers' in system_changes:
        self.nl = NeighborList([rc / 2] * natoms, self_interaction=False, bothways=True )
    
    self.nl.update(self.atoms)

    positions = self.atoms.positions
    cell = self.atoms.cell

    energies = np.zeros(natoms)
    forces = np.zeros((natoms, 3))
    stresses = np.zeros((natoms, 3, 3))

    for ii in range(natoms):
        neighbors, offsets = self.nl.get_neighbors(ii)
        cells = np.dot(offsets, cell)

        # pointing *towards* neighbours
        distance_vectors = positions[neighbors] + cells - positions[ii]

        r = np.sqrt( (distance_vectors ** 2).sum(1) )
        pairwise_energies, pairwise_forces = self.pairwise_e( r )
        pairwise_energies[r > rc] = 0.0
        pairwise_forces[r > rc] = 0.0

        pairwise_forces = pairwise_forces[:, np.newaxis] * distance_vectors
        energies[ii] += 0.5 * pairwise_energies.sum()  # atomic energies
        forces[ii] += pairwise_forces.sum(axis=0)
        stresses[ii] += 0.5 * np.dot(pairwise_forces.T, distance_vectors)

    # no lattice, no stress
    if self.atoms.cell.rank == 3:
        stresses = full_3x3_to_voigt_6_stress(stresses)
        self.results['stress'] = stresses.sum(axis=0) / self.atoms.get_volume()
        self.results['stresses'] = stresses / self.atoms.get_volume()

    energy = energies.sum()
    self.results['energy'] = energy
    self.results['energies'] = energies
    self.results['free_energy'] = energy
    self.results['forces'] = forces

# Introduction

This notebook explains how you can calculate the heat capactiy for a system of interacting particles in a way that incorporates the nuclear quantum effects by exploting the harmonic approximation.  We then use this as a jumping off point for discussing how you can extract dynamic properties from MD simulations.  This part builds on what you learned in the last set of exercises about calculating static properties from MD simulations.

As always please ensure that the key libraries are loaded before you start the exercises by executing the following cell.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Calculating the density of states

In assignments previous to this one you learned two ways to calculate the heat capacity as a function of temperature from molecular dynamics simulations.
When you use the methods that have been introduced thus far you assume that the particles the system is composed of are classical and that the effect of
the quantum nature of matter on the heat capacity is tiny.  In this final assignment we are going to examine this approximation that the particles are classical
in more detail by learning one approach for determining the nuclear quantum effects.

In the approach we will use, we extract properties of the 0 K structure and use these properties to determine the partition function at finite temperature by assuming that the
system behaves as a system of independent quantum harmonic oscillators.  We can derive an exact expression for the partition function of quantum harmonic osciallator and can thus
calculate the properties of such systems at all temperatuures.  Furthermore, the only non-thermodynamic variable that enters the expression for the partition
function of a quantum harmonic oscillator is the oscillators characteristic frequency.  The first thing we will thus need to do is to extract a set of characteristic frequencies from
the 0K structure.  The mathematics that we use to extract these characteristic frequencies is explained in the following video.

If you need some revision on eigenvalues and eigenvectors then you can [look here](https://brilliant.org/wiki/eigenvalues-and-eigenvectors/)

In [1]:
%%HTML
<iframe width="560" height="315" src="https://www.youtube.com/embed/3pm7O70FWUI?si=qGBqXL6ibViNU09k" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" referrerpolicy="strict-origin-when-cross-origin" allowfullscreen></iframe>

We can use ASE to find the 0 K structure of a material by running an optimisation and by the calculating the vibrations.  To do these calculations we need to load some additional parts of ASE using the commands below.

In [ ]:
from ase.optimize import BFGS
from ase.vibrations import Vibrations

We then need to revise what we learned in the previous exercisors and complete
the function `fff` so that it returns the energy for a pair of atoms that are separated by a distance $r_{ij}$ and the force that acts on these atoms when they are separated by this distance.

In [ ]:
def fff(r):
   # Insert your code to calculate the Lennard Jones energy and forces here

   return e, f  # First argument should be energy and second should be force

In [ ]:
runtest(['test_forces1'])

With those initial setup parts done we can now begin to run the optimisation and frequency calculations.  

The optimisation calculation moves the atoms to the positions where they have the lowest energy, which is the position
they should adopt when the system is at 0 K.  To find the minimum energy positions for the atoms in the atom type object `atoms` we can use the following ASE commands:

```python
opt = BFGS(atoms)
opt.run(fmax=0.001)
```

The initial position of the atoms and the potential that acts between them can be set here by using what we learned about ASE in the exercises you completed for the last part of the course.

Once we have found the minimum energy structure we can use ASE to calculate a matrix of second derivatives of the potential (the hessian matrix) by using the following commands:

```python
vib = Vibrations(atoms)
vib.run()
data = vib.get_vibrations()
hessian = data.get_hessian_2d()
```

It is striaghtforward to show that the Hessian matrix has units of one over seconds squared.  The square roots of the eigenvalues of this matrix are thus frequencies.

We can find these eigenvalues and store them in a NumPy array called `eigvals` by using the following commands:

```python
eigvals, eigvecs = np.linalg.eig( hessian )
```

If we do this when the energy is minimised we should find that the eigenvalues are all non-negative.  There should thus be no imaginary frequencies.

__Your task in this exercise is to use the information above about ASE to calculate the frequencies for a system of 108 Lennard Jones particles.__  To complete this exercise you will need to use
the code that you learned last week that generates an initial configuration for the 108 atoms by placing all the atoms on the lattice sites for an FCC crystal.  You will then need to use the
`pairwise_calc` module that I introduced last week to setup ASE so that it uses the function `fff` that you wrote above to calculate the potential energy for any configuration of Lennard Jones particles. 

You should use a cutoff of 4 natural units when computing the Lennard Jones potential.  Hint: setting up the calculator was covered in the exercises on doing MD with multiple particles.

Once you have setup the initial structure and calculator you should be able to use the functions described above for minimising the energy, calculating the hessian and calculating the frequencies that
are described above.

__The task is not completed once you have calculated the frequencies as I would like you to generate a graph that shows the density of states as a function of the frequencies.__  To construct this graph
you will need to place each of the frequencies that you obtained by diagonalising the Hessian matrix into one of the `nbins` histogram bins with centers at `xvals` that I have created for you in `main.py`.
The final lines of codes that I have written should then generate the plot of the density of states for you.

In [ ]:
# Insert code to setup the initial structure here and to setup the function for calculating the potential energy

# Add the code required to do the optimisation that you were given in the instructions

# Add the code from the instructions to calculate the Hessian matrix and the frequencies for the minimised structure
# the final line of this block of code should set the variable called frequencies equal to the frequencies that you
# found from the hessian matrix

# Set this variable equal to the frequencies of the hessian
frequencies =

# This sets up code for calculating a histogram with nbins bins that holds the density of states
# the range of frequncy values starts at slightly less than minfreq and end at slightly more than maxfreq
nbins = 100
minfreq = min(frequencies)
maxfreq = max(frequencies)
delx = 1.01*(maxfreq- minfreq) / nbins
minx = mineng - (maxfreq-minfreq)*0.005

# Find the position of the center of each bin in the histogram
xvals = np.zeros(nbins)
for i in range(nbins) : xvals[i] = minx + (i+0.5)*delx

# You now need to loop over the frequencies that emerged from the hessian and determine how many of these frequencies
# are in each bin of your histogram. The number of frequencies in each bin of the histogram should be accumulated in the
# numpy array called histo below
histo = np.zeros(nbins)


# This will plot the final density of states for you
plt.plot( xvals, histo, 'k-')
plt.xlabel('frequencies / arbitrary units')
plt.ylabel('Number of states')
plt.show()
# This code is required for the Automated feedback, don't delete it!
fighand = plt.gca()

In [ ]:
runtest(['test_calculator1', 'test_density_of_states'])

# Heat capacity for 1D quantum harmonic oscillator

Your task for this exercise is to calculate the heat capcity for a single quantum harmonic oscillator as a function of temperature and to draw a graph of this function.  The following video gives you a short refresher on the statitstical mechanics of the quantum harmonic oscillator.  

If you prefer to read this material you can find some notes [here](https://robust-creature-3df.notion.site/Harmonic-Oscillator-92855b39d13945dabb73cb831dc1234e).

In [2]:
%%HTML
<iframe width="560" height="315" src="https://www.youtube.com/embed/JlGS74_VCX8?si=GdWXZbsrN-PELEVT" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" referrerpolicy="strict-origin-when-cross-origin" allowfullscreen></iframe>

You should implement the code for plotting the heat capacity for a single quantum harmonic oscillator as a function of temperature in the following cell.  When I test your code I assume that the $\hbar \omega$ value for your harmonic oscillator is equal to the variable I have
called `frequency` that is defined below.  

Please ensure that you have a variable called `T` in your final `main.py` code.  This variable should be set equal to a NumPy array that contains the temperatures
at which you evaluated the heat capacity when you generated in the plots.  I use this variable when testing your code.  Please also set the x-axis and y-axis labels
on your graph equal to 'Temperature / natural units' and 'Heat capacity / natural units' respectively.

In [ ]:
frequency = 1.0
T = np.linspace( 0.1, 10, 200 )


# This code is required for the Automated feedback, don't delete it!
fighand = plt.gca()

In [ ]:
runtest(['test_heat_capcity3'])

# Heat capacity for N quantum harmonic oscillators

It is straightforward to adapt what you have learned about calculating the heat capacity for a single quantum harmonic oscillator to calculating the heat capacity
for the solid system that we studied in the previous but one exercise.

At the end of that exercise about calculating the frequencies from the Hessian matrix we had
calculated a vector with 3N elements, where N is the number of atoms.  This vector contained the frequencies of the 3N vibrational modes for the system of N atoms.
Each of these modes is independent so we can thus calculate a partition function for each vibrational mode by substituing the frequency of that mode into the expression
for the partition function of a 1D quantum harmonic oscillator.  The partition function for the N particle system is then the product of these 3N 1D partition functions.

The average energy for a system of 3N harmonic oscillators is thus the sum of the average energies for each of the 3N harmonic oscillator.  The heat capacity is
similarly a sum of the heat capacities of the 3N 1D oscillators.

Given all the above your task is to plot a graph showing the heat capacity per atom as a function of temperature for the three atom system with the 9 frequencies that are
given in the NumPy array called `frequencies`.  You will need to use the expression for the heat capacity of a quantum Harmonic oscillator that you used in the previous exercise
to calculate the heat capacities for each of the frequencies separately.  You will then need to sum all those individual heat capacities in order to (almost) arrive at the final result (remember
I want the heat capacity per atom and there are three atoms).

Please ensure that you have a variable called `T` in your final `main.py` code.  This variable should be set equal to a NumPy array that contains the temperatures
at which you evaluated the heat capacity when you generated in the plots.  I use this variable when testing your code.  Please also set the x-axis and y-axis labels
on your graph equal to 'Temperature / natural units' and 'Heat capacity per atom / natural units' respectively.

_Notice that you can replace the list of frequencies that I have provided you with here with the set of frequencies that you obtained from the second exercise above.  In other words, you can use the set of frequencies that you obtained by diagnalising the Hessian - the test should still pass if you use these frequencies._ 


In [ ]:
frequencies = np.array([1,1,2,2,2,3,4,5,9])
T = np.linspace( 0.1, 30, 200 )


# This code is required for the Automated feedback, don't delete it!
fighand = plt.gca()

In [ ]:
runtest(['test_heat_capcity4'])

# Running constant energy molecular dynamics

In my example report I calculated dynamical properties from molecular dynamics simulations.
When we are calculating these properties we need to be careful about controlling temperature using thermostats.  Thermostats
work by perturbing the velocities of the particles in our simulation box and in doing so can change the measured dynamic properties.

One way to avoid this issue is to run a simulation __without__ a thermostat.  In other words, you can run your simulation in the NVE
rather than the NVT simulation.  You will be running these sorts of simulations to complete the exercises so in this exercise, we are
thus going to learn to run a NVE simulation using ASE.

The procedure that should be followed here is similar to the one that you have follwed in setting up the NVT simulations.  In particular
you need to:

1. Set up an `Atoms` object by calling the `FaceCenteredCubic` method.  As you have done in previous exercises you should run a 3x3x3 cell with a lattice constant of 2^(2/3).
2. Set the masses of all the atoms in your `Atoms` object equal to one.
3. Use the Maxwell Boltzmann distribution to set the velocities of all the atoms in accordance with a temperature of 2.0 natural units.
4. Setup the atoms object to use the `pairwise_calculator` method to calculate the potential energy.  You should use a cutoff of 4 sigma when determining the interactions.  You can use the function called `fff` that you wrote above to calculate the energy and forces on a pair of atoms as you have done in previous exercises.
5. Next you need to set the variable `initial_energy` equal to the total energy of the N particle system you are studying.
6. Once you have done this you are in a position to run the NVE MD using the following commands:

```python
dyn = VelocityVerlet( atoms, tstep )
dyn.run(100)
```

Before running these commands you will need to use what you learned in previous exercises about the `dyn.attach` method to capture the total energy of the system on each timestep as your final task is to generate a graph that shows the
total energy as a function of simulation time.  The labels on the axis of this graph should be 'time / arbitrary units' and 'total energy / arbitrary units'.  The test code here checks that the total energy values that you plot in your
graph are the same as the variable `initial_energy`.  This is a valid test as in NVE MD the total energy of the system should be constant.  In practise (because of cutoffs and the like) energy changes by a small amount in any simulation.
I thus recommend using a small timestep for your MD simulation.

N.B.

_For this exercise I asked you to run a NVE simulation started directly from the input fcc structure.  In practise you would __never__ run a simulation like this.  Before you start running NVE simluations you should run some equilibration
in the NVT ensemble.  These equilibration steps allow the system to equilibrate and get the energy to a reasonable equilibrium value for the temperature of interest.  In other words, you run the equilibration in the NVT ensemble to ensure that
the potential energies of the structures that are being visted are reasonable._


In [ ]:
# You need to setup a very small timestep in order to get the energy to be conserved
tstep = 0.00001

# Insert code from last exercises to create an atoms object and set masses and velocities here.
atoms =

# Attach the method that should be used to calculate energies and forces to the atoms object

# Calculate the initial energy for me here so I can test your code.  Please dont adjust this line
initial_energy = atoms.get_potential_energy() + atoms.get_kinetic_energy()


# And use the ideas that were explained in the instructions file to run 100 steps of constant energy MD.
# Add a method to store the total energy on every step so that you can plot a graph of the energy as a function
# of time.


# This code is required for the Automated feedback, don't delete it!
fighand = plt.gca()

In [ ]:
runtest(['test_calculator2', 'test_conservation'])

# Loading a trajectory

For this exercise and the next one we will be using an NVE trajectory that I have generated for you.  To load the data from this trajectory into your notebook execute the following cell.  Once you have done so you can then move on to the final two exercises, which both use the variable `ftraj` that has been created here.

In [ ]:
from ase.io.trajectory import Trajectory

# This command uses ase to read in the trajectory
ftraj = Trajectory('https://raw.githubusercontent.com/autofeedback-exercises/exercises/main/New-MTH4332/LennardJonesII/nve-short.traj')

# The velocity autocorrelation function

If we have performed a molecular dynamics simulation of a system of $N$ particles we can investigate whether or not the
particles are undergoing an oscillatory motion by calculating the velocity autocorrelation function.  This function is introduced in the following video.

In [3]:
%%HTML
<iframe width="560" height="315" src="https://www.youtube.com/embed/E1a6HokwEzw?si=Bg179C4f2KW_zjK1" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" referrerpolicy="strict-origin-when-cross-origin" allowfullscreen></iframe>

As discussed in the video the autocorrelation function is calculated as follows:

$$
C(\tau) = \langle \mathbf{v}(\tau)\cdot \mathbf{v}(0) \rangle = \frac{1}{3NT(T-\tau)} \sum_{t=0}^{T-\tau} \mathbf{v}(t)\cdot \mathbf{v}(t+\tau)
$$

In this expression $v(t)$ is a vector that contains the velocities of all the particles at time $t$, $T$ is the number of frames in the trajectory and
$N$ is the number of atoms.  The expression above is thus the ensemble average of the dot product between the instantaneous velocities at two times separated by a time interval $\tau$ au.

If the mass of the particles is one the velocity autocorrelation should be equal to the $k_B T$ when $\tau$ equals 0 as it is basically
the sum of the squares of all the velocities divided by the total number of atoms, $N$.  For particles with mass 1 this is double the
kinetic energy and we know that average kinetic energy should be equal to 0.5 $k_B T$ by equipartition.  For $\tau>0$ the autocorrleation
function should decay towards zero. There may, however, be some oscillations in the decay and if these are present that is indicative
of the sorts of oscilations about the lattice sites that you see in a solid.

__Your task in this exercise is to write code to calculate a velocity autocorrelation function.__  You should use the trajectory in the variable `ftraj` that we loaded earlier when calculating this function.  

Remember that you can cycle over all the frames in the trajectory by using a loop like this one:

```python
for atoms in ftraj :
    # Do some analysis on the atoms data in each frame of the trajectory
```

Notice that you can get a 1D vector with $3N$ elements that contain the velocities of all the atoms for the nth frame in the trajectory by using the command below:

```python
vel = ftraj[n].get_velocities().flatten()
```

You can then calculate the dot product that is being averaged in the equation for the autocorrelation function as follows:

```pythnon
dp = np.dot( veln.T, vel ) / len(vel)
```

In the expression above `veln` is a second vector of velocities that is extracted from the trajectory in the same way that the vector `vel` was obtained.

You should calculate the autocorrelation and store its values in the NumPy array called `acf`.  This array has `ncorr` elements.  The 0th element of this array
should contain the velocity autocorrelation for $\tau=0$.  The 1st element should contain the velocity autocorrelation function for frames that are separated by one
timestep.  The 2nd element frames that are separated by two timesteps and so on. To loop and accumulate these averages I would recommend using code like this:

```python
n=0
for atoms in ftraj :
   maxn = min(len(ftraj), n + ncorr) - n
   for i in range(maxn) :
       # Insert code to update the appropriate element of the autocorrelation function here.
```

If you complete the exercise correctly a graph should be generated that shows your estimate of the autocorrelation function.

In [ ]:
# This is number of frames that we are calculating the correlation function over
ncorr = 50



# This sets up arrays to hold the autocorrelation function and the number of estimates
# of each dot product we have
acf, norm = np.zeros(ncorr), np.zeros(ncorr)

# Your code to calculate the autocorrelation function goes here




# This will plot the autocorrelation function.  The value of 0.005 is the time
# between each frame in the trajectory that I have given you here so the xvalues
# are the time lags between the frames for whcih I have calculated the dot product
# of the velocity.
times = 0.005*np.linspace(0, ncorr-1, ncorr)
plt.plot( times, acf, 'k-' )
plt.xlabel('time / arbitrary units')
plt.ylabel('velocity autoccoreation function')
plt.show()
# This code is required for the Automated feedback, don't delete it!
fighand = plt.gca()

In [ ]:
runtest(['test_heat_capcity1'])

# The virbrational density of states

We can get a spectrum showing the vibrational frequencies at a particular temperature by taking the
Fourier transform of the autocorrelation function that you learned to calculate.  Some researchers have
then used the vibrational density of states that emerges from such calculations to calculate heat capacities
in the way that is similar that I showed you for calculating the heat capacity from the eigenvalues of the Hessian.

Calculating the autocorrelation function directly in the way that I showed you in the last exercise hopefully helped you to
understand what this function measures. Researchers typically avoid using this method, however, as it is computationally
expensive and there is a faster way to get the Fourier Transform of the velocity autocorrelation function that uses the
convolution theorem.  In the following video, I thus explain this method for getting the fourier transform of the autocorrelation function (if you ever want the autocorrelation function again you can get this by taking the inverse fourier transform of the function that emerges from the method that I explain in the video.)

If you have forgotten what Fourier Series and Fourier Transforms are you can read recaps of the theory of Fourier Series [here](https://brilliant.org/wiki/fourier-series/) and information on Fourier Transforms [here](https://brilliant.org/wiki/discrete-fourier-transform/).

In [4]:
%%HTML
<iframe width="560" height="315" src="https://www.youtube.com/embed/p2ycReoDyOo?si=qJXtAGrWAjSuz0sM" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" referrerpolicy="strict-origin-when-cross-origin" allowfullscreen></iframe>

Let's now see if we can apply the ideas intorduced in the video to the trajectory that is contained in the variable called `ftraj`.  

Your
first task in calculating the vibrational density of states is to transfer the velocity data in this trajectory into a
matrix that I will call `vtraj` in what follows.  The matrix should have 3N rows, where N is the number of atoms, and M columns,
where M is the number of frames in the trajectory.  The ith column in your matrix should contain the instantaneous velocities that all
N atoms in the system had during at the start of the the ith frame in the trajectory.

Once you have you set all the elements of the matrix you can calculate the spectrum using the following commands:

```python
fftraj = np.fft.rfft(vtraj,axis=1)
fdos = np.mean(fftraj*np.conjugate(fftraj),axis=0) / len(ftraj)
```

There is already code in in the following cell to plot the spectrum for you.  Notice that I have used the function `np.fft.rfftfreq` to generate the
set of frequencies where I have estimates for the spectrum.

In [ ]:
# Your code to calculate the vibrational density of states goes here



# This command generates the xvalues at which your frequencies should be plotted here.
# d here is set equal to the time between adjacent frames in the trajectory
freqvals = np.fft.rfftfreq( len(ftraj), d=0.005 )
# This will generate the graph showing the vibrational density of states for you.
plt.plot( freqvals, fdos, 'k-' )
plt.xlabel('frequency / arbitrary units')
plt.ylabel('vibrational density of states')
plt.show()
# This code is required for the Automated feedback, don't delete it!
fighand = plt.gca()

In [ ]:
runtest(['test_heat_capcity2'])